In [2]:
import numpy as np
import pandas as pd
import xpress as xp


## Data Exploration

In [3]:
# read-data
TransactionsInput = pd.read_excel("231013_Transactions_Input.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [5]:
FuardCases = pd.read_excel("231013_Fraud_Cases.xlsx")
CustomerBase = pd.read_excel("231013_Customer_Base.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [6]:
TransactionsInput["description"].nunique()

NameError: name 'TransactionsInput' is not defined

In [7]:
TransactionsInput

NameError: name 'TransactionsInput' is not defined

In [ ]:
CustomerBase

In [ ]:
TransactionsInput_clean = TransactionsInput.drop('description_prob', axis=1)

In [ ]:
FuardCases

### Find the seasonality

In [3]:
Fuard_time = pd.merge(TransactionsInput,FuardCases, on="transaction_id")[["date","month"]]
Fuard_time['month'] = Fuard_time.apply(lambda row: row['month'] + str(row['date'].year), axis=1)
unique_date, counts_date = np.unique(Fuard_time[['date']], return_counts=True)
unique_month, counts_month = np.unique(Fuard_time[['month']], return_counts=True)
frequency_date = dict(zip(unique_date, counts_date))
frequency_month = dict(zip(unique_month, counts_month))  
print(frequency_date)
print(frequency_month)

NameError: name 'TransactionsInput' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(unique_date,counts_date)

In [ ]:
# TransactionsInput_clean = TransactionsInput_clean[["In_or_Out"] == "paid_in"]

## Create a problem with xpress module

#### The basic model in a single day. (Ignore the learn model)

In [ ]:
# just use for testing
customer_num = 10

### Abstract the test set for testing the code - only use first 10 transactios

In [ ]:
TransactionsInput_clean_test = TransactionsInput_clean[0:10]
TransactionsInput_clean_test

In [ ]:
# data (parameeters) needed in this problem
# NOTE THAT: very strightforward way in this case as this is the idea stage.
# still need to abstract the single instead of a whole data set
x_length = len(TransactionsInput_clean_test["transaction_id"])
y_length = len(TransactionsInput_clean_test["transaction_id"])
Prob_transac = TransactionsInput_clean_test["transac_prob"]
# Prob_description = TransactionsInput_clean_test["description_prob"]  

Customer_test = pd.merge(TransactionsInput_clean_test, CustomerBase, on="customer_id", how="left")[['transaction_id','customer_id','home_bank','customer_prob']]
Prob_customer = Customer_test["customer_prob"]

# bank here is used to distribute the investigation resiponsibility
Bank_to = TransactionsInput_clean_test["bank_to"]
Bank_from = TransactionsInput_clean_test["bank_from"]


# Parameters
Bank = ["A", "B", "C", "D", "E"]
Investigarion_team = [8, 12, 10, 10, 10]

Priority = [1,2,3,4]
Investigation_time = [0.25, 0.5, 1, 2]
Investigation_hire_exter = [40, 60, 100, 150]
Investigation_success = [0.5, 0.6, 0.7, 0.8]


In [ ]:
# test
Customer_test

In [ ]:
# Create a problem called transaction

prob = xp.problem(name='transaction') 

In [ ]:
# Create the x variables
x = xp.var(vartype=xp.binary, name="x") # if we want to investigate the transaction i
f = xp.var(vartype=xp.binary, name="f") # if the in vestigation label transaction i is fraud
y = xp.var(vartype=xp.binary, name="y") #

# Add the variables to the problem
# can only add once. If run the code without cleaning menory, will report error.
prob.addVariable(x,f,y)

### how can i define the shape of variable like xpress

In [ ]:
# add objective value
# expect value on investigation cost
prob.addObjective(i for i in x_length x[i]*)

In [ ]:
# add constraints
prob.addConstraint()